In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('sales_pos.csv')

In [4]:
df1 = df.copy()
df1.shape

(550068, 11)

In [5]:
df1.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


### pivot_table

In [6]:
# prod 별 각 도시의 purchase 평균, 표준편차

pv_df1 = pd.pivot_table(df1, index='prod',columns=['city'],values='purchase',aggfunc=['mean','std'])
pv_df1.head(2)

mean                                      std               \
city                  A             B             C            A            B   
prod                                                                            
P00000142  11077.560784  11174.903448  11150.681818  2283.546514  2280.121291   
P00000242   9945.843373  10377.618785  11282.517857  3681.578007  3028.912919   

                        
city                 C  
prod                    
P00000142  2344.426206  
P00000242  2865.131524

In [7]:
# B 도시에서 구매 평균이 가능 높은 상품은?
pv_df1['mean']['B'].idxmax()

'P00086242'

### multi index : xs()

In [8]:
# df['상위','하위'] 로 selecting  
# pv_df1.xs(('mean','B'),level=[0,1],axis=1)

pv_df1['mean','B'].idxmax()  

# multi index 가 아니면 오류가 난다.df['a','b']

'P00086242'

### stack

In [9]:
# stack : column 을 index 로 이동 
# pivot (columns)-> stack -> groupby (index) 이랑 동일하게 만들수 있네.

pv_df1.stack(level=1)     # level=0 은 mean,std 가 인덱스로 이동

mean          std
prod      city                           
P00000142 A     11077.560784  2283.546514
          B     11174.903448  2280.121291
          C     11150.681818  2344.426206
P00000242 A      9945.843373  3681.578007
          B     10377.618785  3028.912919
...                      ...          ...
P0099842  B      6838.640000  1870.601690
          C      7606.575758  1336.883223
P0099942  A      6686.600000  1463.875609
          B      4712.875000  2072.996826
          C      6883.000000          NaN

[10321 rows x 2 columns]

In [10]:
# 이게 group 이랑 같은 거네.
pd.pivot_table(df1, index=['prod','city'],values='purchase',aggfunc=['mean','std'])

mean          std
                    purchase     purchase
prod      city                           
P00000142 A     11077.560784  2283.546514
          B     11174.903448  2280.121291
          C     11150.681818  2344.426206
P00000242 A      9945.843373  3681.578007
          B     10377.618785  3028.912919
...                      ...          ...
P0099842  B      6838.640000  1870.601690
          C      7606.575758  1336.883223
P0099942  A      6686.600000  1463.875609
          B      4712.875000  2072.996826
          C      6883.000000          NaN

[10321 rows x 2 columns]

In [11]:
# A 도시의 평균, 표준편차 만 

pd.pivot_table(df1, index=['prod','city'],values='purchase',aggfunc=['mean','std']).xs('A',level=1)

,mean,std
,purchase,purchase
prod,,
P00000142,11077.560784,2283.546514
P00000242,9945.843373,3681.578007
P00000342,5205.015625,1816.543041
P00000442,4696.583333,1608.706053
P00000542,4620.159091,1590.366580
...,...,...
P0099442,13718.107692,3758.431493
P0099642,5538.000000,879.767772


### groupby

In [29]:
# 똑 같이 groupby 로 만들어 보자

df1.groupby(by=['prod','city'])[['purchase']].agg(['sum','std'])

purchase             
                    sum          std
prod      city                      
P00000142 A     2824778  2283.546514
          B     4861083  2280.121291
          C     5151615  2344.426206
P00000242 A      825505  3681.578007
          B     1878349  3028.912919
...                 ...          ...
P0099842  B      341932  1870.601690
          C      251017  1336.883223
P0099942  A       33433  1463.875609
          B       37703  2072.996826
          C        6883          NaN

[10321 rows x 2 columns]

In [47]:
# 조건별 groupby : purchase 

# x < 500
sel_pur1 = df1['purchase']<500
# 500<= x < 1000
sel_pur2 = (500<=df1['purchase'])&(df1['purchase']<=1000)
# 10000 < x
sel_pur3 = 1000<df1['purchase']

# Female
sel_Fe = df1['gender']=='F'


cond_group = df1.groupby([sel_pur1,sel_pur2,sel_pur3,sel_Fe])['purchase'].sum()
cond_group

purchase  purchase  purchase  gender
False     False     True      False     3905025334
                              True      1184608058
          True      False     False        3859170
                              True         1383929
True      False     False     False         695596
                              True          240655
Name: purchase, dtype: int64

In [46]:
cond_group[True,False,False]

936251

### unstack

In [13]:
# groupby (index) -> unstack -> pivot_table (columns)
# pd.pivot_table(df1, index='prod',columns=['city'],values='purchase',aggfunc=['mean','std']) 와 같다.

df1.groupby(by=['prod','city'])[['purchase']].agg(['sum','std']).unstack()

purchase                                                  \
                 sum                                std                
city               A          B          C            A            B   
prod                                                                   
P00000142  2824778.0  4861083.0  5151615.0  2283.546514  2280.121291   
P00000242   825505.0  1878349.0  1263642.0  3681.578007  3028.912919   
P00000342   333121.0   526030.0   437324.0  1816.543041  1908.004581   
P00000442   169077.0   192561.0    79535.0  1608.706053  1786.514719   
P00000542   203287.0   400975.0   202950.0  1590.366580  1439.073191   
...              ...        ...        ...          ...          ...   
P0099442    891677.0  1256504.0   722202.0  3758.431493  3613.476528   
P0099642     22152.0    42113.0    19445.0   879.767772  2378.339035   
P0099742    270270.0   386634.0   335044.0  3162.009313  2960.552363   
P0099842    144363.0   341932.0   251017.0  1155.428755  1870.601690   
P0099942     33433.0    37703.0     6883.0  1463.875609  2072.996826   

                        
                        
city                 C  
prod                    
P00000142  2344.426206  
P00000242  2865.131524  
P00000342  1704.636686  
P00000442  1487.392790  
P00000542  2090.943177  
...                ...  
P0099442   3592.391870  
P0099642   2955.543492  
P0099742   2695.186257  
P0099842   1336.883223  
P0099942           NaN  

[3631 rows x 6 columns]

In [19]:
df1.groupby(by=['prod','city'])[['purchase']].agg(['sum','std']).stack()

purchase
prod      city                  
P00000142 A    sum  2.824778e+06
               std  2.283547e+03
          B    sum  4.861083e+06
               std  2.280121e+03
          C    sum  5.151615e+06
...                          ...
P0099942  A    sum  3.343300e+04
               std  1.463876e+03
          B    sum  3.770300e+04
               std  2.072997e+03
          C    sum  6.883000e+03

[20016 rows x 1 columns]

In [126]:
top_prod = df1.groupby(by=['prod'])['purchase'].sum().idxmax()
top_prod

'P00025442'

### condition selectiong

In [140]:
rule1 = df1['prod'].str[1:].astype('int')%4 != 0
df1.loc[rule1,:].groupby(by=['prod'])['purchase'].agg(['sum'])   # ('sum') 하면 Series 리턴

# prod 값이 4의 배수가 아닌 데이터중에서 (rule1) , prod별 purchase 의 합이  .idxmax() 가장 큰 prod ?

,sum
prod,
P00000142,12837476
P00000242,3967496
P00000342,1296475
P00000442,441173
P00000542,807212
...,...
P0099442,2870383
P0099642,83710
P0099742,991948


In [160]:
# A 도시의 평균, 표준편차 만 

rule2 = df1.loc[:,'city'] =='A'
df1.loc[rule2,:].groupby('prod')[['purchase']].agg(['mean','std'])

purchase             
                   mean          std
prod                                
P00000142  11077.560784  2283.546514
P00000242   9945.843373  3681.578007
P00000342   5205.015625  1816.543041
P00000442   4696.583333  1608.706053
P00000542   4620.159091  1590.366580
...                 ...          ...
P0099442   13718.107692  3758.431493
P0099642    5538.000000   879.767772
P0099742    7949.117647  3162.009313
P0099842    7598.052632  1155.428755
P0099942    6686.600000  1463.875609

[3477 rows x 2 columns]

In [127]:
df1.loc[df1['prod'] == top_prod, 'job'].value_counts(normalize=True)

job
4     0.136842
7     0.115789
0     0.110836
17    0.088545
1     0.076780
12    0.072446
14    0.052012
2     0.048297
20    0.046440
16    0.037152
10    0.034056
6     0.033437
15    0.030960
3     0.028483
11    0.023529
5     0.019195
13    0.014861
19    0.012384
18    0.008669
9     0.007430
8     0.001858
Name: proportion, dtype: float64